# Test Benchmark

## Buffer

buffer store function (multi done scenario)

In [ ]:
from agent import AgentSAC
from attrdict import AttrDict
import torch
import yaml
from envs import toy

with open(f'configs/debug.yaml', "r") as stream:
  try:
    cfg = AttrDict(yaml.safe_load(stream))
  except yaml.YAMLError as exc:
    print(exc)
agent = AgentSAC(cfg)

In [ ]:
traj = agent.explore_vec_env()

In [ ]:
agent.buffer.data_parser(agent.buffer.data[8:12], 'info.ag')

In [ ]:
trans = agent.buffer.sample_batch(10, her_rate=1, indices=torch.arange(11, device='cuda:0'))

In [ ]:
obs_dict = agent.EP.obs_parser(trans.state[:10])
info_dict = agent.EP.info_parser(trans.info[:10])
obs_dict.ag, obs_dict.g, info_dict.step, trans.rew

In [ ]:
info_dict = agent.EP.info_parser(trans.info[:20])
info_dict.step

HER buffer

# Wandb

In [ ]:
import wandb
wandb.init(project="debug")

# How to Check Block Distance

In [50]:
import torch
num_envs=2
num_goals=3
min_dist = 1
extra_goals = torch.tensor(
  [
    [
      [[0,0],[0.5,0],[1.5,0]],
      [[0,0],[0.5,0.5],[1.5,0.5]]
    ],
    [
      [[0,0],[1.5,0],[0,1.5]],
      [[3,0],[0,0],[5,0]]
    ],
  ]
)
goal_dist = torch.abs(extra_goals.unsqueeze(-3) - extra_goals.unsqueeze(-2))
satisfied_idx = ((goal_dist[...,0] > min_dist) | \
  (goal_dist[..., 1] > min_dist)) | torch.eye(num_goals).bool()
satisfied_idx = satisfied_idx.all(dim=-1).all(dim=-1)
extra_goals[satisfied_idx].shape

torch.Size([2, 3, 2])

In [17]:
p=0.2
n=3
import math
import numpy as np
np.cumsum([math.factorial(n)/(math.factorial(n-m)*math.factorial(m))*(1-p)**m*p**(n-m) for m in range(n+1)])

array([0.008, 0.104, 0.488, 1.   ])

# Rotate Experience

In [19]:
import torch

grip_pos = torch.tensor([-1,-2,1,-2,2,1])
hand_vel = torch.tensor([0.1,0.1,-0.1,-0.1,-0.1,0.1])
finger_width = torch.tensor([0.01,0.01,0.02,0.02])
block_state = torch.tensor([0]*20)
block_pos = torch.tensor([-10,-20,10,10,20,-10])
goal = torch.tensor([1,2,3,4,5,6])

obs = torch.cat((grip_pos, hand_vel, finger_width, block_state, block_pos, goal))

pos_rot_mat = torch.tensor([
  [-1,0,0],
  [0,-1,0],
  [0,0,1]
])
quat_rot_mat = torch.tensor([
  [0,-1,0,0],
  [1,0,0,0],
  [0,0,0,1],
  [0,0,1,0],
])
block_other_mat = torch.block_diag(*([quat_rot_mat]+[pos_rot_mat]*2))
rot_mat = torch.block_diag(*([pos_rot_mat]*4+[torch.eye(4)]+[block_other_mat]*2+[pos_rot_mat]*4))
print((obs*rot_mat)/obs)

tensor([[-1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0., -1.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ..., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0., -1.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  1.]])
